In [1]:
import pandas as pd
import numpy as np

In [2]:
# preprocess data
classes = []
classes.extend(range(0,57))
classes.append("target")

with open('./spambase.train', 'r') as f:
    data = f.readlines()
train = []
for row in data:
    train.append([eval(i) for i in row.split(",")])
train_data = pd.DataFrame(train, columns = classes)
all_data = train_data.copy(deep=True)

with open('./spambase.test', 'r') as f:
    data = f.readlines()
test = []
for row in data:
    test.append([eval(i) for i in row.split(",")])
test_data = pd.DataFrame(test, columns = classes)

full = all_data.append(test_data)

for i in range(0, 57):
    
    train_data.loc[train_data[i] < full[i].median(), i] = 1
    train_data.loc[train_data[i] >= full[i].median(), i] = 2
    test_data.loc[test_data[i] < full[i].median(), i] = 1
    test_data.loc[test_data[i] >= full[i].median(), i] = 2

C:\Users\kongj\AppData\Local\Temp\ipykernel_24052\3764687825.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full = all_data.append(test_data)


In [3]:
# probability of spam
spam = 0
for value in train_data["target"]:
    if value == 1:
        spam = spam + 1

probSpam = spam/len(train_data["target"])

In [4]:
# calculate P(X = x | Y = y) for each class x, and spam/not spam y
listProbs = []
for column in range(57):
    oneGivenSpam = 0
    oneGivenNotSpam = 0
    twoGivenSpam = 0
    twoGivenNotSpam = 0
    for i in range(0,len(train_data)):
        if train_data[column][i] == 1:
            if train_data["target"][i] == 1:
                oneGivenSpam = oneGivenSpam + 1
            else:
                oneGivenNotSpam = oneGivenNotSpam + 1
        else:
            if train_data["target"][i] == 1:
                twoGivenSpam = twoGivenSpam + 1
            else:
                twoGivenNotSpam = twoGivenNotSpam + 1
                
    listProbs.append([oneGivenSpam/spam, twoGivenSpam/spam, oneGivenNotSpam/(2000-spam), twoGivenNotSpam/(2000-spam)])
    
# listProbs = list of probabilities for each class

In [5]:
predict = []
# predict each row of test whether it is spam or not
for i in range(0, len(test_data)):
    # multiply P(Y = y) with product of P(X = x | Y = y) to get P(Y = y | X = x)
    probSpam = spam/len(train_data["target"])
    probNotSpam = 1 - probSpam
    for j in range(0, 57):
        if test_data.iloc[i][j] == 1:
            
            probSpam = probSpam * listProbs[j][0]
            probNotSpam = probNotSpam * listProbs[j][2]
            
        else:
            
            probSpam = probSpam * listProbs[j][1]
            probNotSpam = probNotSpam * listProbs[j][3]
    # if P(Y = 'spam' | X = x) > P(Y = 'not spam' | X = x), predict spam
    if (probSpam > probNotSpam):
        
        predict.append(1)
        
    else:
        
        predict.append(0)
        

In [6]:
correct = 0
for i in range(0, len(predict)):
    if predict[i] == test_data.iloc[i]["target"]:
        correct = correct + 1

print("Test Error: ")
print(1 - (correct/len(test_data)))

Test Error: 
0.261437908496732


In [7]:
correct = 0
for i in range(0, len(predict)):
    if test_data.iloc[i]["target"] == 0:
        correct = correct + 1

print("Sanity Test Error: ")        
print(1 - (correct/len(test_data)))

Sanity Test Error: 
0.3856209150326797
